In [1]:
import torch
from torchvision.datasets import mnist # 导入 pytorch 内置的 mnist 数据

from torch import nn
from torch.autograd import Variable

In [2]:
# 使用内置函数下载 mnist 数据集
train_set = mnist.MNIST('./data', train=True, download=True)
test_set = mnist.MNIST('./data', train=False, download=True)


In [3]:
import numpy as np
a_data, a_label = train_set[0]
a_data = np.array(a_data, dtype='float32')
print(a_data.shape)

(28, 28)


In [4]:
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 标准化，这个技巧之后会讲到
    x = x.reshape((-1,)) # 拉平
    x = torch.from_numpy(x)
    return x

train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True) # 重新载入数据集，申明定义的数据变换
test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)

In [5]:
from torch.utils.data import DataLoader
# 使用 pytorch 自带的 DataLoader 定义一个数据迭代器
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

In [6]:
a, a_label = next(iter(train_data))
# 使用 Sequential 定义 4 层神经网络
net = nn.Sequential(
    nn.Linear(784, 400),
    nn.ReLU(),
    nn.Linear(400, 200),
    nn.ReLU(),
    nn.Linear(200, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
)
net

Sequential(
  (0): Linear(in_features=784, out_features=400, bias=True)
  (1): ReLU()
  (2): Linear(in_features=400, out_features=200, bias=True)
  (3): ReLU()
  (4): Linear(in_features=200, out_features=100, bias=True)
  (5): ReLU()
  (6): Linear(in_features=100, out_features=10, bias=True)
)

In [7]:
# 定义 loss 函数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 1e-1) # 使用随机梯度下降，学习率 0.1

In [8]:
# Train
losses = []
acces = []
eval_losses = []
eval_acces = []

for e in range(20):
    train_loss = 0
    train_acc = 0
    net.train()
    for im, label in train_data:
        im = Variable(im)
        label = Variable(label)
        # 前向传播
        out = net(im)
        loss = criterion(out, label)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 记录误差
        train_loss += loss.item()
        # 计算分类的准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        train_acc += acc
        
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    # 在测试集上检验效果
    eval_loss = 0
    eval_acc = 0
    net.eval() # 将模型改为预测模式
    for im, label in test_data:
        im = Variable(im)
        label = Variable(label)
        out = net(im)
        loss = criterion(out, label)
        # 记录误差
        eval_loss += loss.item()
        # 记录准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
        
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Eval Loss: {:.6f}, Eval Acc: {:.6f}'
          .format(e, train_loss / len(train_data), train_acc / len(train_data), 
                     eval_loss / len(test_data), eval_acc / len(test_data)))

epoch: 0, Train Loss: 0.501213, Train Acc: 0.834871, Eval Loss: 0.218925, Eval Acc: 0.929391
epoch: 1, Train Loss: 0.169393, Train Acc: 0.946862, Eval Loss: 0.129550, Eval Acc: 0.957773
epoch: 2, Train Loss: 0.116965, Train Acc: 0.963136, Eval Loss: 0.184991, Eval Acc: 0.937500
epoch: 3, Train Loss: 0.094204, Train Acc: 0.970566, Eval Loss: 0.114190, Eval Acc: 0.963509
epoch: 4, Train Loss: 0.074499, Train Acc: 0.976712, Eval Loss: 0.114613, Eval Acc: 0.964201
epoch: 5, Train Loss: 0.065042, Train Acc: 0.979244, Eval Loss: 0.083570, Eval Acc: 0.973794
epoch: 6, Train Loss: 0.053576, Train Acc: 0.982776, Eval Loss: 0.245770, Eval Acc: 0.927512
epoch: 7, Train Loss: 0.044384, Train Acc: 0.985491, Eval Loss: 0.073522, Eval Acc: 0.977947
epoch: 8, Train Loss: 0.042377, Train Acc: 0.986191, Eval Loss: 0.071717, Eval Acc: 0.977255
epoch: 9, Train Loss: 0.034203, Train Acc: 0.988739, Eval Loss: 0.103164, Eval Acc: 0.969937
epoch: 10, Train Loss: 0.028180, Train Acc: 0.990922, Eval Loss: 0.069